# Question 15.2

In the videos, we saw the “diet problem”. (The diet problem is one of the first large-scale optimization
problems to be studied in practice. Back in the 1930’s and 40’s, the Army wanted to meet the nutritional
requirements of its soldiers while minimizing the cost.) In this homework you get to solve a diet problem with real data. The data is given in the file diet.xls. 

2.Please add to your model the following constraints (which might require adding more variables) and solve the new model:
a.If a food is selected, then a minimum of 1/10 serving must be chosen. (Hint: now you will need two variables for each food i: whether it is chosen, and how much is part of the diet. You’ll also need to write a constraint to link them.)
b.Many people dislike celery and frozen broccoli. So at most one, but not both, can be selected.
c.To get day-to-day variety in protein, at least 3 kinds of meat/poultry/fish/eggs must be selected. [If something is ambiguous (e.g., should bean-and-bacon soup be considered meat?), just call it whatever you think is appropriate – I want you to learn how to write this type of constraint, but I don’t really care whether we agree on how to classify foods!]


In [1]:
#Importing necesary libraries
from pulp import *
import pandas as pd

In [3]:
#Getting the data
data=pd.read_excel('diet.xls')

# Data Cleaning

Let's first clean the data to make it analysis ready

In [4]:
#Viewing the data
data.head()

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,Frozen Broccoli,0.16,10 Oz Pkg,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,"Carrots,Raw",0.07,1/2 Cup Shredded,23.7,0.0,0.1,19.2,5.6,1.6,0.6,15471.0,5.1,14.9,0.3
2,"Celery, Raw",0.04,1 Stalk,6.4,0.0,0.1,34.8,1.5,0.7,0.3,53.6,2.8,16.0,0.2
3,Frozen Corn,0.18,1/2 Cup,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
4,"Lettuce,Iceberg,Raw",0.02,1 Leaf,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1


In [5]:
data.isnull().sum()

Foods              3
Price/ Serving     3
Serving Size       1
Calories           1
Cholesterol mg     1
Total_Fat g        1
Sodium mg          1
Carbohydrates g    1
Dietary_Fiber g    1
Protein g          1
Vit_A IU           1
Vit_C IU           1
Calcium mg         1
Iron mg            1
dtype: int64

In [241]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Foods            64 non-null     object 
 1   Price/ Serving   64 non-null     float64
 2   Serving Size     66 non-null     object 
 3   Calories         66 non-null     float64
 4   Cholesterol mg   66 non-null     float64
 5   Total_Fat g      66 non-null     float64
 6   Sodium mg        66 non-null     float64
 7   Carbohydrates g  66 non-null     float64
 8   Dietary_Fiber g  66 non-null     float64
 9   Protein g        66 non-null     float64
 10  Vit_A IU         66 non-null     float64
 11  Vit_C IU         66 non-null     float64
 12  Calcium mg       66 non-null     float64
 13  Iron mg          66 non-null     float64
dtypes: float64(12), object(2)
memory usage: 7.5+ KB


In [7]:
#Separating the constraint table from the main data
constraints_table=data.iloc[65:,2:].reset_index(drop=True)

In [8]:
#Viewing constraints
constraints_table

,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,Minimum daily intake,1500.0,30.0,20.0,800.0,130.0,125.0,60.0,1000.0,400.0,700.0,10.0
1,Maximum daily intake,2500.0,240.0,70.0,2000.0,450.0,250.0,100.0,10000.0,5000.0,1500.0,40.0


In [9]:
#Data used for optimization
data=data.iloc[0:64,:]

In [12]:
#Now the data table contains only the variables without the constraints
data.head()

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,Frozen Broccoli,0.16,10 Oz Pkg,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,"Carrots,Raw",0.07,1/2 Cup Shredded,23.7,0.0,0.1,19.2,5.6,1.6,0.6,15471.0,5.1,14.9,0.3
2,"Celery, Raw",0.04,1 Stalk,6.4,0.0,0.1,34.8,1.5,0.7,0.3,53.6,2.8,16.0,0.2
3,Frozen Corn,0.18,1/2 Cup,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
4,"Lettuce,Iceberg,Raw",0.02,1 Leaf,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Foods            64 non-null     object 
 1   Price/ Serving   64 non-null     float64
 2   Serving Size     64 non-null     object 
 3   Calories         64 non-null     float64
 4   Cholesterol mg   64 non-null     float64
 5   Total_Fat g      64 non-null     float64
 6   Sodium mg        64 non-null     float64
 7   Carbohydrates g  64 non-null     float64
 8   Dietary_Fiber g  64 non-null     float64
 9   Protein g        64 non-null     float64
 10  Vit_A IU         64 non-null     float64
 11  Vit_C IU         64 non-null     float64
 12  Calcium mg       64 non-null     float64
 13  Iron mg          64 non-null     float64
dtypes: float64(12), object(2)
memory usage: 7.1+ KB


# Solution

In [247]:
# Creating the problem variable to contain the problem data. We will use the minimize function for the problem
var=LpProblem('Diet_Problem', LpMinimize)

In [248]:
# Creating dictionaries for all the varibles

cost=dict(zip(data['Foods'],data['Price/ Serving']))
Calories=dict(zip(data['Foods'],data['Calories']))
Cholesterol=dict(zip(data['Foods'],data['Cholesterol mg']))
Total_Fat=dict(zip(data['Foods'],data['Total_Fat g']))
Sodium=dict(zip(data['Foods'],data['Sodium mg']))
Carbohydrates=dict(zip(data['Foods'],data['Carbohydrates g']))
Dietary_Fiber=dict(zip(data['Foods'],data['Dietary_Fiber g']))
Protein=dict(zip(data['Foods'],data['Protein g']))
Vit_A =dict(zip(data['Foods'],data['Vit_A IU']))
Vit_C =dict(zip(data['Foods'],data['Vit_C IU']))
Calcium=dict(zip(data['Foods'],data['Calcium mg']))
Iron=dict(zip(data['Foods'],data['Iron mg']))

In [249]:
# A dictionary called variables is created to contain the referenced Variables
variable = LpVariable.dicts("Food",data['Foods'],0,cat='Continuous')

In [250]:
# A dictionary called selected is created to contain the selected Variable
selected = LpVariable.dicts("Select",data['Foods'],0,1,cat='Integer')

In [251]:
#Creating objective functions
var+=lpSum([cost[i]*variable[i] for i in data['Foods']])

In [253]:
#Adding Calories constraints
var+=lpSum([Calories[j]*variable[j] for j in data['Foods']])>=constraints_table.iloc[0,1]
var+=lpSum([Calories[j]*variable[j] for j in data['Foods']])<=constraints_table.iloc[1,1]

#Adding Cholesterol constraints
var+=lpSum([Cholesterol[j]*variable[j] for j in data['Foods']])>=constraints_table.iloc[0,2]
var+=lpSum([Cholesterol[j]*variable[j] for j in data['Foods']])<=constraints_table.iloc[1,2]

#Adding Total_Fat constraints
var+=lpSum([Total_Fat[j]*variable[j] for j in data['Foods']])>=constraints_table.iloc[0,3]
var+=lpSum([Total_Fat[j]*variable[j] for j in data['Foods']])<=constraints_table.iloc[1,3]

#Adding Sodium constraints
var+=lpSum([Sodium[j]*variable[j] for j in data['Foods']])>=constraints_table.iloc[0,4]
var+=lpSum([Sodium[j]*variable[j] for j in data['Foods']])<=constraints_table.iloc[1,4]

#Adding Carbohydrates constraints
var+=lpSum([Carbohydrates[j]*variable[j] for j in data['Foods']])>=constraints_table.iloc[0,5]
var+=lpSum([Carbohydrates[j]*variable[j] for j in data['Foods']])<=constraints_table.iloc[1,5]

#Adding Dietary_Fiber constraints
var+=lpSum([Dietary_Fiber[j]*variable[j] for j in data['Foods']])>=constraints_table.iloc[0,6]
var+=lpSum([Dietary_Fiber[j]*variable[j] for j in data['Foods']])<=constraints_table.iloc[1,6]

#Adding Protein constraints
var+=lpSum([Protein[j]*variable[j] for j in data['Foods']])>=constraints_table.iloc[0,7]
var+=lpSum([Protein[j]*variable[j] for j in data['Foods']])<=constraints_table.iloc[1,7]

#Adding Vit_A constraints
var+=lpSum([Vit_A[j]*variable[j] for j in data['Foods']])>=constraints_table.iloc[0,8]
var+=lpSum([Vit_A[j]*variable[j] for j in data['Foods']])<=constraints_table.iloc[1,8]

#Adding Vit_A constraints
var+=lpSum([Vit_C[j]*variable[j] for j in data['Foods']])>=constraints_table.iloc[0,9]
var+=lpSum([Vit_C[j]*variable[j] for j in data['Foods']])<=constraints_table.iloc[1,9]

#Adding Calcium constraints
var+=lpSum([Calcium[j]*variable[j] for j in data['Foods']])>=constraints_table.iloc[0,10]
var+=lpSum([Calcium[j]*variable[j] for j in data['Foods']])<=constraints_table.iloc[1,10]

#Adding Iron constraints
var+=lpSum([Iron[j]*variable[j] for j in data['Foods']])>=constraints_table.iloc[0,11]
var+=lpSum([Iron[j]*variable[j] for j in data['Foods']])<=constraints_table.iloc[1,11]

In [254]:
#Adding new constraints
#1) a.If a food is selected, then a minimum of 1/10 serving must be chosen. 
#Hint: now you will need two variables for each food i: whether it is chosen, 
#and how much is part of the diet. You’ll also need to write a constraint to link them.)

for k in data['Foods']:
    var += variable[k]>= selected[k]*0.1
    var += variable[k]<= selected[k]*1e5

In [255]:
#b.Many people dislike celery and frozen broccoli. So at most one, but not both, can be selected.

var+=selected['Frozen Broccoli']+selected['Celery, Raw']<=1

In [256]:
#c.To get day-to-day variety in protein, at least 3 kinds of meat/poultry/fish/eggs must be selected. 
choices = ['Beanbacn Soup,W/Watr','Bologna,Turkey','Frankfurter, Beef','Ham,Sliced,Extralean',
                  'Hamburger W/Toppings','Hotdog, Plain','Kielbasa,Prk','Neweng Clamchwd','Pizza W/Pepperoni',
                  'Poached Eggs','Pork','Roasted Chicken','Sardines in Oil','Scrambled Eggs','Vegetbeef Soup',
                   'White Tuna in Water']

In [257]:
var += lpSum([selected[i] for i in choices]) >= 3.0

In [258]:
# The problem data is written to an .lp file
var.writeLP("DietProblem2.lp")

[Food_2%_Lowfat_Milk,
 Food_3.3%_Fat,Whole_Milk,
 Food_Apple,Raw,W_Skin,
 Food_Apple_Pie,
 Food_Bagels,
 Food_Banana,
 Food_Beanbacn_Soup,W_Watr,
 Food_Bologna,Turkey,
 Food_Butter,Regular,
 Food_Cap'N_Crunch,
 Food_Carrots,Raw,
 Food_Celery,_Raw,
 Food_Cheddar_Cheese,
 Food_Cheerios,
 Food_Chicknoodl_Soup,
 Food_Chocolate_Chip_Cookies,
 Food_Corn_Flks,_Kellogg'S,
 Food_Couscous,
 Food_Crm_Mshrm_Soup,W_Mlk,
 Food_Frankfurter,_Beef,
 Food_Frozen_Broccoli,
 Food_Frozen_Corn,
 Food_Grapes,
 Food_Ham,Sliced,Extralean,
 Food_Hamburger_W_Toppings,
 Food_Hotdog,_Plain,
 Food_Kielbasa,Prk,
 Food_Kiwifruit,Raw,Fresh,
 Food_Lettuce,Iceberg,Raw,
 Food_Macaroni,Ckd,
 Food_Malt_O_Meal,Choc,
 Food_New_E_Clamchwd,W_Mlk,
 Food_Neweng_Clamchwd,
 Food_Oatmeal,
 Food_Oatmeal_Cookies,
 Food_Oranges,
 Food_Peanut_Butter,
 Food_Peppers,_Sweet,_Raw,
 Food_Pizza_W_Pepperoni,
 Food_Poached_Eggs,
 Food_Popcorn,Air_Popped,
 Food_Pork,
 Food_Potato_Chips,Bbqflvr,
 Food_Potatoes,_Baked,
 Food_Pretzels,
 Food_Raisi

In [259]:
# The problem is solved using PuLP's choice of Solver
var.solve()

1

In [260]:
# The status of the solution is printed to the screen
print("Status:", LpStatus[var.status])

Status: Optimal


In [261]:
# Each of the variables is printed with it's resolved optimum value
for v in var.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)

Food_Celery,_Raw = 42.399358
Food_Kielbasa,Prk = 0.1
Food_Lettuce,Iceberg,Raw = 82.802586
Food_Oranges = 3.0771841
Food_Peanut_Butter = 1.9429716
Food_Poached_Eggs = 0.1
Food_Popcorn,Air_Popped = 13.223294
Food_Scrambled_Eggs = 0.1
Select_Celery,_Raw = 1.0
Select_Kielbasa,Prk = 1.0
Select_Lettuce,Iceberg,Raw = 1.0
Select_Oranges = 1.0
Select_Peanut_Butter = 1.0
Select_Poached_Eggs = 1.0
Select_Popcorn,Air_Popped = 1.0
Select_Scrambled_Eggs = 1.0


In [262]:
# The optimised objective function value is printed to the screen
print("Total Cost of Balanced diet =${}".format(round(value(var.objective),2)))

Total Cost of Balanced diet =$4.51
